In [1]:
from i10_bluesky.devices.rasor.detectorSlits import *
from i10_bluesky.devices.scalerCard import ScaleCard
from i10_bluesky.devices.pimteAD import *
from i10_bluesky.plans.rasor.pimte_plans import AdPlan
from ophyd_async.core import DeviceCollector 
from ophyd_async.core.detector import TriggerInfo, DetectorTrigger
from bluesky.callbacks.best_effort import BestEffortCallback
from bluesky import RunEngine
from bluesky.plans import count, scan 
import bluesky.plan_stubs
from ophyd.sim import det1
from bluesky import utils
from bluesky.utils import Msg
from dodal.beamlines.beamline_utils import set_directory_provider
from ophyd_async.core._providers import StaticDirectoryProvider
import asyncio
loop = asyncio.get_event_loop()
bec = BestEffortCallback()
RE = RunEngine({})


In [3]:
df =  StaticDirectoryProvider("/dls/i10/data/2024/cm37251-2/processing/", "i10-")

with DeviceCollector():
       pimte = HDFStatsPimte(
        "BL10I-EA-PIMTE-01:",
        df,
        "pimte",
    )

loop.create_task(pimte.connect())

<Task pending name='Task-152' coro=<Device.connect() running at /venv/lib/python3.11/site-packages/ophyd_async/core/device.py:61>>

In [4]:
result = loop.create_task(pimte.drv.acquire_time.read())

In [5]:
result

<Task finished name='Task-297' coro=<SignalR.read() done, defined at /venv/lib/python3.11/site-packages/ophyd_async/core/signal.py:28> result={'pimte-drv-acquire_time': {'alarm_severity': 0, 'timestamp': 1712155927.093704, 'value': 0.003}}>

In [6]:
RE(count([pimte]),bec)



Transient Scan ID: 1     Time: 2024-04-03 15:35:49
Persistent Unique Scan ID: '6e6dd8fa-ed02-4971-b350-fabb7a5efc7b'
New stream: 'primary'


/venv/lib/python3.11/site-packages/bluesky/callbacks/core.py:332: UserWarning: The key pimte will be skipped because LiveTable does not know how to display the dtype array
  warnings.warn("The key {} will be skipped because LiveTable "


+-----------+------------+
|   seq_num |       time |
+-----------+------------+
|         1 | 15:35:55.8 |
+-----------+------------+
generator count ['6e6dd8fa'] (scan num: 1)





('6e6dd8fa-ed02-4971-b350-fabb7a5efc7b',)

In [7]:
def takeImg(det,value):
    yield Msg("stage", det)
    yield Msg('prepare', det, value)
    yield Msg("unstage", det)
ti = TriggerInfo(1, DetectorTrigger.internal,6,0.003)

In [8]:
def takeImg2(det,value):
    yield Msg("stage", det)
    yield Msg("trigger", det)
    yield Msg("unstage", det)

In [44]:
RE(takeImg2(pimte,ti))

In [11]:
RE(takeImg(pimte,ti))

()

In [10]:
pimtePlan = AdPlan(pimte)


In [31]:
RE(AdPlan.tiggerImg(pimte,ti))

()

In [10]:
RE(AdPlan.tiggerImg(pimtePlan.det,pimtePlan._getTriggerInfo()))

()

In [15]:
RE(pimtePlan.takeImg())

NameError: name 'md' is not defined

In [11]:
RE(pimtePlan.takeImg(exposure = 0.003, n_img = 3))

()

In [13]:
asyncio.create_task(pimte.writer.open())

<Task pending name='Task-556' coro=<HDFWriter.open() running at /venv/lib/python3.11/site-packages/ophyd_async/epics/areadetector/writers/hdf_writer.py:43>>

In [22]:
asyncio.wait_for(pimte.trigger(), timeout =10)

<coroutine object wait_for at 0x7f6359b1b890>

In [33]:
asyncio.gather(pimte.writer.close(), pimte.controller.disarm())

<_GatheringFuture pending>

In [43]:
result = asyncio.create_task(test())

In [41]:
result

<Task finished name='Task-929' coro=<test() done, defined at /tmp/ipykernel_138034/2493302126.py:1> exception=RuntimeError('asyncio.run() cannot be called from a running event loop')>

In [16]:
pimte.writer.close

<bound method HDFWriter.close of <ophyd_async.epics.areadetector.writers.hdf_writer.HDFWriter object at 0x7f5bfe008290>>